In [1]:
import os
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import datetime
import importlib
import global_settings
importlib.reload(global_settings)
import statsmodels.api as sm
from sklearn.preprocessing import scale

In [2]:
# Define working folder

setts = global_settings.Settings()

root_folder = setts.root_folder
tmp1_folder = setts.tmp1_folder
tmp2_folder = setts.tmp2_folder
output_folder = setts.output_folder

# tmp1_folder = root_folder + "/data/tmp"
# tmp2_folder = root_folder + "/data/tmp2"
# input_folder = root_folder + "/input"
# output_folder = root_folder + "/output"

# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)
    
year_limit = setts.year_limit

Directory found: D:\Software\temp\CeDX\data_2024_05/data
Directory found: D:\Software\temp\CeDX\data_2024_05/data/tmp
Directory found: D:\Software\temp\CeDX\data_2024_05/data/tmp2
Directory found: D:\Software\temp\CeDX\data_2024_05/output


# Add Customer Info from sale

In [3]:
import re
import difflib
import unicodedata

def replace_tp(tp):
    tp = tp.replace('THANH HOÁ', 'THANH HÓA')
    tp = tp.replace('THÀNH PHỐTHANH HÓA', 'THANH HÓA')
    tp = tp.replace('KHÁNH HOÀ', 'KHÁNH HÒA')
    tp = tp.replace('TP.HCM', 'HỒ CHÍ MINH')
    tp = tp.replace('TPHCM', 'HỒ CHÍ MINH')
    tp = tp.replace('TP HCM', 'HỒ CHÍ MINH')
    tp = tp.replace('HCM', 'HỒ CHÍ MINH')
    tp = tp.replace('HỒ CHÍ MINH', 'HỒ CHÍ MINH')
    tp = tp.replace('DAKLAK', 'ĐẮK LẮK')
    tp = tp.replace('ÐẮK LẮK', 'ĐẮK LẮK')
    tp = tp.replace('ÐỒNG NAI', 'ĐỒNG NAI')
    tp = tp.replace('ÐỒNG NAI', 'ĐỒNG NAI')
    tp = tp.replace('ÐÀ NẴNG', 'ĐÀ NẴNG')
    tp = tp.replace('HOÀ BÌNH', 'HÒA BÌNH')
    tp = tp.replace('THỪA THIÊN HUẾ', 'THỪA THIÊN - HUẾ')
    tp = tp.replace('BÌNH ÐỊNH', 'BÌNH ĐỊNH')
    tp = tp.replace('ÐỒNG THÁP', 'ĐỒNG THÁP')
    tp = tp.replace('HN', 'HÀ NỘI')
    tp = tp.replace('HÀ NỘI', 'HÀ NỘI')
    tp = tp.replace('HUẾ', 'THỪA THIÊN - HUẾ')
    tp = tp.replace('APP', '')
    tp = tp.replace('(', '')
    tp = tp.replace(')', '')
    #tp = tp.replace('Ð', 'Đ')
    #tp = tp.replace('ÒA', 'OÀ')
    #tp = tp.replace('OÀ', 'ÒA')
    #tp = re.sub(r"[^a-zA-Z0-9]+", ' ', tp)
    return tp

def no_accent_vietnamese(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵà]', 'a', s)
    s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪÀ]', 'A', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡộ]', 'o', s)
    s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠỘ]', 'O', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(r'[Đ]', 'D', s)
    s = re.sub(r'[đ]', 'd', s)
    s = re.sub(r"[^a-zA-Z0-9]+", ' ', s)
    return s

# def no_accent_vietnamese(input_string):
#     normalized_string = ''.join(
#         unicodedata.normalize('NFD', char)
#         for char in input_string
#         if unicodedata.category(char) != 'Mn'
#     )
#     return normalized_string





def replace_city(df, city_col_in, city_col_out):
    vn_city = (
    "Cà Mau",
    "Bạc Liêu",
    "Sóc Trăng",
    "Hậu Giang",
    "Cần Thơ",
    "Kiên Giang",
    "An Giang",
    "Đồng Tháp",
    "Vĩnh Long",
    "Trà Vinh",
    "Bến Tre",
    "Tiền Giang",
    "Long An",
    "Hồ Chí Minh",
    "Bà Rịa",
    "Vũng Tàu",
    "Bà Rịa - Vũng Tàu",
    "Đồng Nai",
    "Bình Dương",
    "Tây Ninh",
    "Bình Phước",
    "Lâm Đồng",
    "Đắk Nông",
    "Đắk Lắk",
    "Gia Lai",
    "Kon Tum",
    "Bình Thuận",
    "Ninh Thuận",
    "Khánh Hòa",
    "Phú Yên",
    "Bình Định",
    "Quảng Ngãi",
    "Quảng Nam",
    "Đà Nẵng",
    "Huế",
    "Quảng Trị",
    "Quảng Bình",
    "Hà Tĩnh",
    "Nghệ An",
    "Thanh Hóa",
    "Ninh Bình",
    "Nam Định",
    "Hà Nam",
    "Thái Bình",
    "Hưng Yên",
    "Hải Phòng",
    "Hải Dương",
    "Hà Nội",
    "Bắc Ninh",
    "Vĩnh Phúc",
    "Phú Thọ",
    "Bắc Giang",
    "Quảng Ninh",
    "Lạng Sơn",
    "Thái Nguyên",
    "Hòa Bình",
    "Yên Bái",
    "Sơn La",
    "Lai Châu",
    "Điện Biên",
    "Lào Cai",
    "Tuyên Quang",
    "Bắc Kạn",
    "Cao Bằng",
    "Hà Giang",
    )
    city_master_list = vn_city
    address_list = tuple(df[city_col_in])
    thanhpho_list = []
    for add in address_list:
        has_city = ''
        add = add.upper()
        add = re.sub(' +', ' ', add).strip()
        add_og = add
        for thanh_pho in city_master_list:
            add = replace_tp(add)
            if add[-len(thanh_pho):] == thanh_pho.upper():
                has_city = thanh_pho.upper()
                break

        if has_city == '':
            for thanh_pho in city_master_list:
                temp = difflib.SequenceMatcher(None, no_accent_vietnamese(add_og), no_accent_vietnamese(thanh_pho).upper())
                if temp.ratio() >= 0.9:
                    has_city = thanh_pho.upper()
                    break
                else:
                    if no_accent_vietnamese(thanh_pho).upper() in no_accent_vietnamese(add_og).upper():
                        has_city = thanh_pho.upper()
                        break

        thanhpho_list.append(has_city)
    df[city_col_out] = thanhpho_list

    df[city_col_out] = df[city_col_out].apply(lambda x: x.replace('Bà Rịa - Vũng Tàu'.upper(), 'BRVT'.upper())
                                  .replace('Bà Rịa'.upper(), 'BRVT'.upper())
                                  .replace('Vũng Tàu'.upper(), 'BRVT'.upper()))

    df[city_col_out] = df[city_col_out].apply(lambda x: x.replace( 'BRVT'.upper(), 'Bà Rịa - Vũng Tàu'.upper()))
    return df

In [4]:
def remove_non_numeric_chars(input_string):
    return ''.join(char for char in input_string if char.isdigit())

def modify_phone_num_column(df, col):
    df[col] = df[col].apply(lambda x: remove_non_numeric_chars(str(x))).dropna()
    df = df[df[col] != '']
    df[col] = df[col].astype(np.longlong).astype(str)
    return df

In [5]:
sale = pd.read_csv(root_folder + '/data/CeDX_Sale.csv')
sale['GENDER'] = sale['MALE'].astype('str').str.replace('X', 'MALE') + sale['FEMALE'].astype('str').str.replace('X', 'FEMALE')
sale['GENDER'] = sale['GENDER'].astype(str).str.strip().str.replace('nan', '')
sale['CITY'] = sale['CITY'].astype(str)
sale = replace_city(sale, 'CITY', 'CITY')

print(sale['GENDER'])
print(sale['CITY'])

C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_22884\3644766933.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sale = pd.read_csv(root_folder + '/data/CeDX_Sale.csv')


0           MALE
1           MALE
2           MALE
3           MALE
4               
           ...  
234400      MALE
234401      MALE
234402      MALE
234403      MALE
234404    FEMALE
Name: GENDER, Length: 234405, dtype: object
0           LẠNG SƠN
1             HÀ NỘI
2             HÀ NỘI
3             HÀ NỘI
4             HÀ NỘI
             ...    
234400    BÌNH DƯƠNG
234401     THANH HÓA
234402     VĨNH PHÚC
234403     VĨNH LONG
234404     HẢI PHÒNG
Name: CITY, Length: 234405, dtype: object


In [6]:
sale = sale[['MOBILE_PHONE', 'CUSTOMER_NAME', 'GENDER', 'CITY']]
sale = sale.drop_duplicates(subset=['MOBILE_PHONE'])
sale = modify_phone_num_column(sale, 'MOBILE_PHONE')

dataset = pd.read_csv(output_folder + '/dataset_all_scored.csv')
dataset['UID'] = dataset['UID'].astype(str)
dataset = pd.merge(dataset, right=sale, how='left', left_on= 'UID',right_on='MOBILE_PHONE').drop(columns=['MOBILE_PHONE'], axis=1)
del sale
dataset.to_excel(output_folder + '/dataset_all_scored_info.xlsx', index=False)

# Customer Segmenting

In [7]:
data_full_direct = output_folder + "/dataset_all_full_columns.csv"
data_scored_direct = output_folder + "/dataset_all_scored.csv"

df_scored = pd.read_csv(data_scored_direct)[['UID', 'Score']]
df = pd.read_csv(data_full_direct)#[['UID', setts.month_to_str, 'visit_freq_y', 'TTL']]
df = pd.merge(df, right=df_scored, how='left', on='UID')

df = df.rename(columns = { 'visit_freq_y' : 'Frequency', 'TTL' : 'Revenue'})
# df['Recency']=-df[setts.month_to_str] #last month vua tinh

# df['F']=pd.qcut(df['Frequency'], 5, labels = range(1,6), duplicates='drop').astype(int) # frequency= visit_freq_y
# df['M']=pd.qcut(df['Revenue'], 5, labels = range(1,6), duplicates='drop').astype(int)  #monetary = TTL
# df['Recency']=pd.qcut(df['Recency'], 5, labels = range(1,6), duplicates='drop').astype(int)

# df['Frequency+Monetary']=(df['F']+df['M'])/2
# df['Frequency+Monetary']=df['Frequency+Monetary'].round(0)

# df['RFM']= df['Recency'].astype(str)+ df['Frequency+Monetary'].apply(lambda x: str(x)[0])

customers = {'55' : 'Champions',
'54' : 'Champions',
'45' : 'Loyal Customers',
'44' : 'Loyal Customers',
'35' : 'Loyal Customers',
'34' : 'Loyal Customers',
'53' : 'Potential Loyalist',
'52' : 'Potential Loyalist',
'43' : 'Potential Loyalist',
'42' : 'Potential Loyalist',
'51' : 'New Customers',
'41' : 'Promising',
'31' : 'Need Attention',
'32' : 'Need Attention',
'33' : 'About to Sleep',
'25' : "Can't lose them",
'15' : "Can't lose them",
'24' : 'At Risk',
'23' : 'At Risk',
'14' : 'At Risk',
'13' : 'At Risk',
'22' : 'Hibernating',
'21' : 'Hibernating',
'12' : 'Hibernating',
'11' : 'Hibernating'}

def classify_customer(customer_number):
    if customer_number in customers:
        return customers[customer_number]
    else:
        return 'Unknown'

def seg_conditions(month_to, PM2ov, Score, total_buy):
    if month_to < 7 and PM2ov == 1 and Score < 0.6 and total_buy == 1:
        #return 'Temporary loyal customer'
        return 'C2 Significant customer'
    elif month_to < 7 and PM2ov == 1 and Score >= 0.6 and total_buy == 1:
        #return 'Potential loyal customer'
        return 'C1 Valued customer'
    elif month_to < 7 and PM2ov == 1 and Score < 0.6 and total_buy > 1:
        #return 'Passive Loyal customer'
        return 'D2 Promise customer'
    elif month_to < 7 and PM2ov == 1 and Score >= 0.6 and total_buy > 1:
        #return 'Honda Fan / Loyal customer'
        return 'D1 Honda Fan'
    elif month_to < 7 and PM2ov == 0 and Score < 0.6:
        #return 'Normal customer'
        return 'B2 Sensitive customer'
    elif month_to < 7 and PM2ov == 0 and Score >= 0.6:
        #return 'Potential customer'
        return 'B1 Potential customer'
    elif month_to >= 7 and Score < 0.6:
        #return 'Lost customer'
        return 'A2 Lost customer'
    elif month_to >= 7 and Score >= 0.6:
        return 'A1 Attention customer'
        #return 'Almost lost customer'
    else:
        return 'Unknown'
    
def segment_customer(df, cot):
    output = []
    for index in range(len(df)):
        row = df.iloc[index]
        month_to = row[cot]
        PM2ov = row['PM2ov']
        Score = row['Score']
        total_buy = row['total_buy']
        #PM_vnum = row['PM_vnum']
        output.append(seg_conditions(month_to, PM2ov, Score, total_buy))
    return output
    
df['Customer segmentation'] = segment_customer(df, setts.month_to_str)
#df['Customer segmentation'] = .apply(segment_customer(df, setts.month_to_str), axis=1)
#df['Customer type']=df['RFM'].apply(lambda x:classify_customer(x))

df.to_csv(output_folder + "/dataset_all_RFM.csv", index=False)

In [8]:
rfm_path = output_folder + "/dataset_all_RFM.csv"
info_path = output_folder + "/dataset_all_scored_info.xlsx"

#df_rfm = pd.read_csv(rfm_path)[['UID', 'Customer type','Customer segmentation']]
df_rfm = pd.read_csv(rfm_path)[['UID','Customer segmentation']]
df_info = pd.read_excel(info_path)

df_info = pd.merge(df_info, df_rfm, how='left', on='UID')
df_info.to_excel(output_folder + '/dataset_all_scored_info_RFM_result.xlsx', index=False)

# Extract customer action data

In [22]:
# df_final = df_final.drop_duplicates()
# df_final.to_csv(output_folder + "/customer_action_service_final.csv", index=False)

In [23]:
# df_final_backup = df_final.copy()